<a href="https://colab.research.google.com/github/sdikici/Demand_Forecasting_Prophet_DeepAR/blob/main/DeepAR_hyperparameter_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d sercandikici/merged-dataset-electricty-weather-for-modelling
! unzip merged-dataset-electricty-weather-for-modelling.zip

Dataset URL: https://www.kaggle.com/datasets/sercandikici/merged-dataset-electricty-weather-for-modelling
License(s): unknown
  0% 0.00/104k [00:00<?, ?B/s]
100% 104k/104k [00:00<00:00, 78.7MB/s]
Archive:  merged-dataset-electricty-weather-for-modelling.zip
  inflating: merged_data.csv         


In [3]:
!pip install gluonts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.5 MB/s eta 0:00:00


In [4]:
pip install mxnet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 32.0 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [5]:
pip install mxnet --upgrade

In [6]:
import numpy as np
np.bool = np.bool_

In [7]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon
import json
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx import DeepAREstimator
from gluonts.mx.trainer import Trainer
import itertools

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [8]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true))
    return mape

def root_mean_squared_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def r_squared(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mean_y_true = np.mean(y_true)
    ss_total = np.sum((y_true - mean_y_true) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_residual / ss_total)
    return r2

In [9]:
mx.random.seed(7)
np.random.seed(7)

In [10]:
df_merged = pd.read_csv("merged_data.csv")
df_merged['settlement_date'] = pd.to_datetime(df_merged['settlement_date'])

In [11]:
df_model_1 = df_merged[["tsd","settlement_date"]]
df_model_1= df_model_1.set_index("settlement_date")
split_from = 90*12 #train test split is from 90days

train_data = df_model_1[:-split_from]
test_data = df_model_1[-split_from:]

In [12]:
freq = "2H"
days_to_predict=7
prediction_length = days_to_predict*12 #predicting 7 days
context_length = prediction_length *2

In [13]:
# Define the parameter grid for DeepAR
param_grid = {
    'epochs': [5, 8, 10],
    'learning_rate': [0.0001, 0.001],
    'num_cells': [40, 64],
    'num_layers': [2, 4]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mapes = []  # Store the MAPEs for each parameter combination here

# Use cross validation to evaluate all parameters
for params in all_params:
    # Define DeepAR estimator with given parameters
    estimator = DeepAREstimator(
        freq=freq,
        context_length=context_length,
        prediction_length=prediction_length,
        num_layers=params['num_layers'],
        num_cells=params['num_cells'],
        trainer=Trainer(
            epochs=params['epochs'],
            learning_rate=params['learning_rate']
        )
    )

    subset = test_data[test_data.index <= "2023-10-10 02:00:00"]

    train_ds = ListDataset(
    [{"start":train_data.index[0],
      "target":train_data.tsd}],
    freq=freq
)

    test_ds = ListDataset(
        [{"start":subset.index[0],
          "target":subset.tsd}],
        freq=freq
    )


    # Train the model
    m = estimator.train(training_data=train_ds)

    # Generate forecasts
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=m,
        num_samples=100
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    # Get MAPE for the forecasts
    evaluator = Evaluator()
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts))
    mape = agg_metrics["MAPE"]
    mapes.append(mape)

# Combine parameters and corresponding MAPEs into a DataFrame
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
print(tuning_results)


100%|██████████| 50/50 [00:30<00:00,  1.62it/s, epoch=5/5, avg_epoch_loss=9.18]
Running evaluation: 1it [00:00,  7.61it/s]
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
100%|██████████| 50/50 [00:47<00:00,  1.06it/s, epoch=5/5, avg_epoch_loss=8.99]
Running evaluation: 1it [00:00,  6.76it/s]
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: 

    epochs  learning_rate  num_cells  num_layers      mape
0        5         0.0001         40           2  1.258105
1        5         0.0001         40           4  0.297282
2        5         0.0001         64           2  0.682995
3        5         0.0001         64           4  0.339104
4        5         0.0010         40           2  0.304882
5        5         0.0010         40           4  0.202300
6        5         0.0010         64           2  0.220839
7        5         0.0010         64           4  0.191164
8        8         0.0001         40           2  0.227985
9        8         0.0001         40           4  0.407961
10       8         0.0001         64           2  0.611744
11       8         0.0001         64           4  0.212004
12       8         0.0010         40           2  0.220546
13       8         0.0010         40           4  0.179961
14       8         0.0010         64           2  0.179285
15       8         0.0010         64           4  0.2267


/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/astype.py:138: UserWarning: Warning: converting a masked element to nan.
  return arr.astype(dtype, copy=True)


In [14]:
tuning_results

,epochs,learning_rate,num_cells,num_layers,mape
0,5,0.0001,40,2,1.258105
1,5,0.0001,40,4,0.297282
2,5,0.0001,64,2,0.682995
3,5,0.0001,64,4,0.339104
4,5,0.0010,40,2,0.304882
5,5,0.0010,40,4,0.202300
6,5,0.0010,64,2,0.220839
7,5,0.0010,64,4,0.191164
8,8,0.0001,40,2,0.227985
9,8,0.0001,40,4,0.407961
